## Imports

In [ ]:
# builtin
from itertools import combinations

# pmlb
from pmlb import fetch_data

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone

# matplotlib
import matplotlib.pyplot as plt

# seaborn
import seaborn as sns

# numpy
import numpy as np

## Load Pollen dataset

In [ ]:
pollen = fetch_data("529_pollen")
X, y = fetch_data("529_pollen", return_X_y=True)

## Preliminary data analysis
Looking into the pollen dataset, the target are real numbers, between -12.0391 to 10.8673.
There are 3848 examples in it.
The dataset contains synthetic pollen created by David Coleman at RCA Laboratories in Princeton. The "ellipsoidal voids" in the pollen were simulated, and you can find more information on how this was done on the openml source page. Although the distribution used to generate the data is Gaussian, it has been modified by scaling, rotating, and translating the data. The variable names in the dataset are not based on any actual pollen attributes.

### features
* RIDGE - continuous. lengths of grain in dimension x.
* NUB - continuous. lengths of grain in dimension y.
* CRACK - continuous. lengths of grain in dimension z.
* WEIGHT - continuous. 

Acocording to info we can verify that there are 3848 examples, and each of our 4 features and target is of type float, and that all values are not None.

In [ ]:
pollen.info()

From the describe() function, we can learn the range, mean and standart deviation and some more on our features and target.

In [ ]:
pollen.describe()

We are plotting all of the histograms between each 2 features, in order to see if we encounter anything ineresting.
As we can see, most of the data are gaussian distributed and we can't find any corellation between 2 features.
As we didn't find any connection between 2 features, we will continue with the current data.

In [ ]:
sns.pairplot(pollen, hue='target', markers='*')

## Learning using LinearRegression
Now we are going to learn using the LinearRegression model.
We are going to use a test-train ration of 0.1,0.2,...,0.9 to train a linear regressor.
We will evaluate the absolute error of the model on the train set and then we will plot the accuracy of each ratio.

In [ ]:
rs = np.arange(0.1,1,0.1)

In [ ]:
def run_linear_regressor(rs: list[float], X: np.ndarray, y: np.ndarray) -> list[float]:
    errors = []
    for r in rs:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=r)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X_train, y_train)
        mae = mean_absolute_error(y_test, linear_regressor.predict(X_test))
        print(f"{r:.01f}: {mae:.04f}")
        errors.append(mae)
    return errors

In [ ]:
def plot_errors(rs: list[float], errors: list[float]) -> None:
    plt.plot(rs, errors, marker='o')
    plt.grid(True)
    plt.xlabel("Train Size Ration")
    plt.ylabel("Mean Absolute Error on Test set")
    plt.title('Mean Absolute Error of LinearRegression Model on Different Test Set Ratios')
    plt.show()

In [ ]:
errors = run_linear_regressor(rs, X, y)

In [ ]:
plot_errors(rs, errors)

### Naive k-features selection
We're going to run the LinearRegression model, each time with k features from the data.

Then we're going to compare the k features that achieved the best results on the test set.

We're going to use k=2 and k=3

In [ ]:
def naive_k_feature_selection(train: tuple, test: tuple, k: int):
    X_train, y_train = train
    X_test, y_test = test
    
    mae_dict = {}
    
    # Get all combinations of k elements
    comb = list(combinations(range(X_train.shape[1]), k))
    linear_regressor = LinearRegression()
    for features in comb:
        curr_model = clone(linear_regressor)
        curr_X_train = X_train[:, features]
        curr_X_test = X_test[:, features]
        curr_model.fit(curr_X_train, y_train)
        mae = mean_absolute_error(y_test, curr_model.predict(curr_X_test))
        mae_dict[features] = mae
    
    best_features = max(mae_dict, key=mae_dict.get)
    return best_features, mae_dict[best_features]
    

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
features = list(pollen.keys()).remove('target')
# for 2 features
k = 2
best_features, mae = naive_k_feature_selection((X_train, y_train), (X_test, y_test), k)
print(f"Best features for k={k}: features={[pollen.columns[i] for i in best_features]}, mean absolute error={mae:.04f}")

# for 3 features
k = 3
best_features, mae = naive_k_feature_selection((X_train, y_train), (X_test, y_test), k)
print(f"Best features for k={k}: features={[pollen.columns[i] for i in best_features]}, mean absolute error={mae:.04f}")
